In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install tensorflow

In [2]:
main_path = r'.'
data_path = main_path+'/data'
import sys
sys.path.append(main_path)
from BayDS.lib.pipeline import *
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union
from BayDS.lib.io import *
from BayDS.lib.training import *
import os
import pickle

## Merging original features and best features

In [2]:
data = pd.read_pickle('./temp/data.pkl')

In [3]:
best_features = pd.read_parquet('./FE2/Corr_features_3155.pqt')


In [4]:
best_features['userid'] = data['userid']


In [5]:
for i in range(30):
    best_features[f'field{i}'] = data[f'field{i}']
    best_features[f'goal1'] = data[f'goal1']
for i in range(1,6):
    best_features[f'indicator_goal2{i}'] = data[f'indicator_goal2{i}']
    best_features[f'goal2{i}'] = data[f'goal2{i}']

In [10]:
del data
gc.collect()

0

In [6]:
for col in best_features.columns:
    if best_features[col].dtypes.name == 'object':
        best_features[col] = pd.Categorical(best_features[col])

In [7]:
best_features.to_parquet("./FE2/BEST3155+Orig.pqt")

In [9]:
best_features.index

Int64Index([1000000, 1000001, 1000002, 1000003, 1000004, 1000005, 1000006,
            1000007, 1000008, 1000009,
            ...
             455001,  455002,  455003,  455004,  455005,  455006,  455007,
             455008,  455009,  455010],
           dtype='int64', name='orderid', length=651067)

## Training


In [9]:
import dask.dataframe as dd
data = pd.read_parquet("FE2/BEST3155+Orig.pqt")

In [11]:
test_idx = data[data['goal1']<0].index
test=data.loc[test_idx]
del data
gc.collect()

6886

In [18]:
data = pd.read_pickle("FE/initial_data.pkl")

In [3]:
exclude_features_from_training = ['indicator_goal21', 'indicator_goal22', 'indicator_goal23',
       'indicator_goal24', 'indicator_goal25', 'goal21', 'goal22', 'goal23','field12',
       'goal24', 'goal25', 'goal1','userid', 'order_till_hour', 'order_till_weekday', 'order_till_month', 'lang_country']

In [14]:
all_features = test.columns
train_features = [f for f in all_features if f not in exclude_features_from_training]
X_test = test.loc[:,train_features]

In [16]:
from sklearn.model_selection import GroupKFold, KFold

NFOLDS = 5

params = {
          "verbosity": -1,
          'num_leaves': 491,
#           'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': 4,
          'learning_rate': 0.005,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
#           'reg_alpha': 0.3899927210061127,
#           'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }
train_options = {
        "model_type":'lgb',
        "params": params,
        "eval_metric":'auc',
        'early_stopping_rounds': 300,
        'n_estimators': 100,
        'averaging': 'rank',
        'use_groups': False,
    }

In [26]:
result_dict = {}
prediction = np.zeros((len(X_test), 1))
oof = {}
scores=[]
feature_importance = pd.DataFrame()
for ifold in range(5):
    print(f'Fold {ifold + 1} started at {time.ctime()}')
    X_train = pd.read_parquet(f'./AGGCV/X_train_agg_fold_{ifold}.pqt').loc[:,train_features]
    y_train = data.goal1.loc[X_train.index].astype(np.uint8)
    X_valid = pd.read_parquet(f'./AGGCV/X_valid_agg_fold_{ifold}.pqt').loc[:,train_features]
    y_valid = data.goal1.loc[X_valid.index].astype(np.uint8)

    model = lgb.LGBMClassifier(**params, n_estimators=5000, n_jobs=-1)
    model.fit(X_train, y_train,
      eval_set=[(X_train, y_train), (X_valid, y_valid)],
      eval_metric='auc',
      verbose=100, early_stopping_rounds=300)
    gc.collect()
    y_pred_valid = model.predict_proba(X_valid, num_iteration=model.best_iteration_)[:, 1]

    scores.append(fast_auc(y_valid, y_pred_valid))
    fold_importance = pd.DataFrame()
    fold_importance["feature"] = train_features
    fold_importance["importance"] = model.feature_importances_
    fold_importance["fold"] = ifold + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
    y_pred = model.predict_proba(X_test)[:, 1]
    oof[ifold] = y_pred_valid.reshape(-1, 1)
    scores.append(fast_auc(y_valid, y_pred_valid))
    if X_test is not None:
        prediction += pd.Series(y_pred).rank().values.reshape(-1, 1)

Fold 1 started at Tue Jan 14 23:47:03 2020
Training until validation scores don't improve for 300 rounds
[100]	training's auc: 0.713063	valid_1's auc: 0.67858
[200]	training's auc: 0.728697	valid_1's auc: 0.688508
[300]	training's auc: 0.740183	valid_1's auc: 0.694216
[400]	training's auc: 0.751323	valid_1's auc: 0.698492
[500]	training's auc: 0.760444	valid_1's auc: 0.700272
[600]	training's auc: 0.768679	valid_1's auc: 0.702266
[700]	training's auc: 0.776048	valid_1's auc: 0.704936
[800]	training's auc: 0.78236	valid_1's auc: 0.706804
[900]	training's auc: 0.78811	valid_1's auc: 0.708538
[1000]	training's auc: 0.793355	valid_1's auc: 0.709588
[1100]	training's auc: 0.798403	valid_1's auc: 0.710791
[1200]	training's auc: 0.802817	valid_1's auc: 0.711194
[1300]	training's auc: 0.807214	valid_1's auc: 0.711597
[1400]	training's auc: 0.811403	valid_1's auc: 0.711799
[1500]	training's auc: 0.815252	valid_1's auc: 0.71201
[1600]	training's auc: 0.819177	valid_1's auc: 0.712382
[1700]	train

In [27]:
prediction /= 5
score = sum(scores)/len(scores)        
imp = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
            by="importance", ascending=True)

In [30]:
prediction

array([[ 76799.9],
       [147681.9],
       [448131.2],
       ...,
       [152081.8],
       [413899. ],
       [320131. ]])

In [28]:
###SAVE RESULTS
import datetime
t = datetime.datetime.now().strftime("%m-%d-%H-%M")
test['proba'] = prediction
sub = pd.read_csv('data/onetwotrip_challenge_sub1.csv')
sub['proba'] = test['proba'] / test['proba'].max()
sub.reset_index(drop=True).set_index('orderid').to_csv(f'temp/submission-{t}.csv')

import pickle
pickle.dump(oof, open(f'temp/oof-kfold-{t}.pkl','wb'))